In [172]:
import os, json, re, string, requests, pprint, nltk, operator, numpy as np, math

In [153]:
all_words = []

def read_file(file_path):
    with open (file_path) as file:
        json_content = json.load(file)
        item_count = 0
        for item in json_content:
            item_count += 1
            text_content = re.sub("<.*?>", "", item["textContent"])
            text_content = text_content.replace('-\n', '')
            word_content = text_content.split()
            topicSpecificPunctuation = '„”–§…«»'
            translator = str.maketrans('', '', string.punctuation+topicSpecificPunctuation)
            
            for word in word_content:
                word = word.translate(translator).lower()
                if len(word)>0:
                    all_words.append(word)
                
#         print(item_count)
        
def read_all_judgments_from_2018():
    for filename in os.listdir("../data_filtered/"):
#         print(filename)
        read_file("../data_filtered/" + filename)
        
read_all_judgments_from_2018()
print(len(all_words))
TOTAL_COUNT = len(all_words)

2530124


In [138]:
data_to_query = (",").join(all_words[:1000])
r = requests.post(data=data_to_query.encode("utf-8"), url="http://localhost:9200")
response_text = r.text
splited_response = response_text.splitlines()
splited_response = [" ".join(x.replace("\t", " ").replace("none", "")[1:].split(":")[:2][:1]).replace(" ", ":")
                    for x in splited_response if ":" in x]

In [141]:
bigrams = nltk.bigrams(splited_response)
unigrams_frequency = nltk.FreqDist(splited_response)
bigrams_frequency = nltk.FreqDist(bigrams)

In [148]:
sorted_unigrams = sorted(unigrams_frequency.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(sorted_unigrams[:30])

[('w:prep', 96),
 ('z:prep', 48),
 ('rok:brev', 30),
 ('dzień:subst', 28),
 ('celny:adj', 16),
 ('październik:subst', 15),
 ('na:prep', 14),
 ('koszt:subst', 13),
 ('tytuł:subst', 12),
 ('kwota:subst', 12),
 ('podatek:subst', 11),
 ('luty:adj', 11),
 ('kwiecień:subst', 11),
 ('urząd:subst', 11),
 ('dyrektor:subst', 11),
 ('izba:subst', 11),
 ('koło:brev', 10),
 ('akcyzowy:adj', 10),
 ('6:adj', 10),
 ('2:num', 10),
 ('wierzytelność:subst', 9),
 ('decyzja:subst', 9),
 ('naczelnik:subst', 9),
 ('to:subst', 9),
 ('administracyjny:adj', 8),
 ('marzec:subst', 8),
 ('maj:subst', 8),
 ('złoty:brev', 8),
 ('egzekucja:subst', 7),
 ('wrzesień:subst', 7)]


In [149]:
sorted_bigrams = sorted(bigrams_frequency.items(), key=operator.itemgetter(1), reverse=True)
pprint.pprint(sorted_bigrams[:30])

[(('z:prep', 'dzień:subst'), 21),
 (('celny:adj', 'w:prep'), 16),
 (('izba:subst', 'celny:adj'), 11),
 (('podatek:subst', 'akcyzowy:adj'), 10),
 (('dzień:subst', '6:adj'), 10),
 (('dyrektor:subst', 'izba:subst'), 10),
 (('w:prep', 'rok:brev'), 10),
 (('naczelnik:subst', 'urząd:subst'), 9),
 (('w:prep', 'koło:brev'), 8),
 (('z:prep', 'tytuł:subst'), 7),
 (('luty:adj', 'marzec:subst'), 7),
 (('marzec:subst', 'kwiecień:subst'), 7),
 (('kwiecień:subst', 'maj:subst'), 7),
 (('maj:subst', 'wrzesień:subst'), 7),
 (('wrzesień:subst', 'październik:subst'), 7),
 (('w:prep', 'z:prep'), 7),
 (('skarbowy:adj', 'w:prep'), 7),
 (('w:prep', 'postać:subst'), 6),
 (('nienależnie:adv', 'pobrać:ppas'), 6),
 (('pobrać:ppas', 'podatek:subst'), 6),
 (('urząd:subst', 'skarbowy:adj'), 6),
 (('koło:brev', '2:num'), 6),
 (('rok:brev', 'w:prep'), 6),
 (('akcyzowy:adj', 'za:prep'), 5),
 (('październik:subst', '2011:adj'), 5),
 (('2011:adj', 'rok:brev'), 5),
 (('urząd:subst', 'celny:adj'), 5),
 (('egzekucyjny:adj',

In [176]:
def shannon_entrophy(word_occurences, total_words):
    sum = 0
    for x in np.nditer(word_occurences):
        if x!= 0:
            sum += (x/total_words) * math.log(x/total_words)
    return sum

def H(word_occurences):
    return shannon_entrophy(word_occurences, TOTAL_COUNT)

def calculate_contingency_table(bigram, bigram_count, total_words):
    first, second = bigram[0], bigram[1]
    first_occurence, second_occurence = unigrams_frequency[first], unigrams_frequency[second]
    '''
    |------  |---------| 
    | A,B    |B,notA   |
    |------  |---------|
    | A,notB |notA,notB|
    |------|---------|
    '''
    return np.array([
            [bigram_count, first_occurence-bigram_count],
            [second_occurence-bigram_count, total_words-first_occurence-second_occurence-bigram_count]])

def log_likeliheood_ratio(bigram_key):
    k = calculate_contingency_table(bigram_key, bigrams_frequency[bigram_key], TOTAL_COUNT)
    return 2 * np.sum(k) * (H(k) - H(k.sum(axis=0)) -H(k.sum(axis=1)))

In [181]:
log_ratios = [(key, log_likeliheood_ratio(key), value) for key, value in bigrams_frequency.items()]

In [186]:
sorted_by_log_ratios = sorted(log_ratios, key=operator.itemgetter(1), reverse=True)
sorted_by_log_ratios[:30]

[(('z:prep', 'dzień:subst'), 520.07778548639874, 21),
 (('celny:adj', 'w:prep'), 392.5644569745964, 16),
 (('izba:subst', 'celny:adj'), 317.73148199571244, 11),
 (('podatek:subst', 'akcyzowy:adj'), 302.1191375150101, 10),
 (('dyrektor:subst', 'izba:subst'), 295.41719714139975, 10),
 (('dzień:subst', '6:adj'), 272.32299976393369, 10),
 (('naczelnik:subst', 'urząd:subst'), 269.40470764313113, 9),
 (('maj:subst', 'wrzesień:subst'), 215.14047478312713, 7),
 (('w:prep', 'rok:brev'), 206.47698130782291, 10),
 (('luty:adj', 'marzec:subst'), 200.71995871307902, 7),
 (('marzec:subst', 'kwiecień:subst'), 200.71995871307902, 7),
 (('kwiecień:subst', 'maj:subst'), 200.71995871307902, 7),
 (('wrzesień:subst', 'październik:subst'), 200.4411794202536, 7),
 (('nienależnie:adv', 'pobrać:ppas'), 191.42319687215652, 6),
 (('w:prep', 'koło:brev'), 185.54757815430102, 8),
 (('pobrać:ppas', 'podatek:subst'), 176.26506551876258, 6),
 (('skarbowy:adj', 'w:prep'), 171.03420455620736, 7),
 (('urząd:subst', 'ska

In [210]:
filtered_by_tags = [log_ratio for log_ratio in log_ratios 
                    if "subst" in log_ratio[0][0] and
                    ("subst" in log_ratio[0][1] or "adj" in log_ratio[0][1])]
filtered_by_tags_sorted = sorted(filtered_by_tags, key=operator.itemgetter(1), reverse=True)
filtered_by_tags_sorted[:30]

[(('izba:subst', 'celny:adj'), 317.73148199571244, 11),
 (('podatek:subst', 'akcyzowy:adj'), 302.1191375150101, 10),
 (('dyrektor:subst', 'izba:subst'), 295.41719714139975, 10),
 (('dzień:subst', '6:adj'), 272.32299976393369, 10),
 (('naczelnik:subst', 'urząd:subst'), 269.40470764313113, 9),
 (('maj:subst', 'wrzesień:subst'), 215.14047478312713, 7),
 (('marzec:subst', 'kwiecień:subst'), 200.71995871307902, 7),
 (('kwiecień:subst', 'maj:subst'), 200.71995871307902, 7),
 (('wrzesień:subst', 'październik:subst'), 200.4411794202536, 7),
 (('urząd:subst', 'skarbowy:adj'), 170.52346824680001, 6),
 (('październik:subst', '2011:adj'), 142.24733339468855, 5),
 (('urząd:subst', 'celny:adj'), 126.30990637406356, 5),
 (('miesiąc:subst', 'luty:adj'), 116.43885197279674, 4),
 (('dzień:subst', '8:adj'), 107.89296419580768, 4),
 (('decyzja:subst', 'naczelnik:subst'), 106.12888660232015, 4),
 (('koszt:subst', 'egzekucyjny:adj'), 105.25050823935115, 4),
 (('zwrot:subst', 'koszt:subst'), 105.250508239351